# Example

In [1]:
import numpy as np

# patch for backward compatibility
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_
    
import gymnasium as gym

env = gym.make("Ant-v4", render_mode="human")  # replace with HalfCheetah-v4 or Walker2d-v4
obs = env.reset()

for _ in range(1000):
    env.render()
    action = env.action_space.sample()  # random action
    obs, reward, terminated, truncated, info = env.step(action)
    
    done = terminated or truncated
    if done:
        obs = env.reset()

env.close()

/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(
/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


In [2]:
print(env.observation_space)

Box(-inf, inf, (27,), float64)


In [3]:
print(env.action_space)

Box(-1.0, 1.0, (8,), float32)


# PID

In [17]:
import mujoco
import numpy as np
import mediapy as media
from scipy.spatial.transform import Rotation as R

# --- Load model and create simulation ---
model = mujoco.MjModel.from_xml_path("ant.xml")
data = mujoco.MjData(model)

# --- Gait parameters ---
t = 0.0
dt = model.opt.timestep
amplitude = 0.5
freq = 1
phase_offsets = np.array([0, np.pi, np.pi, 0, np.pi, 0, 0, np.pi])
# PID gains
Kp = 5.0  # proportional
Kd = 1.0  # derivative


# --- Video / renderer setup ---
height, width = 320, 320
n_seconds = 10
framerate = 30
n_frames = n_seconds * framerate
frames = []

# --- Offscreen rendering ---
with mujoco.Renderer(model, height, width) as renderer:
    for i in range(n_frames):
        # Step simulation to match video frame timing
        while data.time * framerate < i:
            # Get current torso angles and angular velocities
            # Get root quaternion (w, x, y, z)
            quat = data.qpos[3:7]

            # Convert to Euler angles (roll, pitch, yaw)
            r = R.from_quat([quat[1], quat[2], quat[3], quat[0]])  # Note: [x, y, z, w]
            roll, pitch, yaw = r.as_euler('xyz', degrees=False)
            roll_vel, pitch_vel, _ = data.qvel[3:6]

            # Compute stabilizing torques
            tau_roll  = -Kp*roll  - Kd*roll_vel
            tau_pitch = -Kp*pitch - Kd*pitch_vel

            # Apply to corresponding joints (replace with your actuator indices)
            # Assuming first two actuators control torso orientation (adjust if needed)
            data.ctrl[0] += tau_roll
            data.ctrl[1] += tau_pitch
            # Open-loop sine-wave control
            data.ctrl[2:] = amplitude * np.sin(2 * np.pi * freq * t + phase_offsets[2:])
            mujoco.mj_step(model, data)
            t += dt

        # Render current frame
        renderer.update_scene(data)  # use default camera
        frame = renderer.render()
        frames.append(frame)

# --- Play video ---
media.show_video(frames, fps=framerate)

This browser does not support the video tag.


# COM

In [18]:
import mujoco
import numpy as np
import mediapy as media

# --- Load model ---
model = mujoco.MjModel.from_xml_path("ant.xml")
data = mujoco.MjData(model)

# --- Open-loop gait ---
t = 0.0
dt = model.opt.timestep
amplitude = 1.0
freq = 0.5
phase_offsets = np.array([0, np.pi, np.pi, 0, np.pi, 0, 0, np.pi])

# --- Video setup ---
height, width = 240, 320
n_seconds = 6
framerate = 30
n_frames = n_seconds * framerate
frames = []

# PD gains for CoM stabilization
Kp = 15.0
Kd = 5.0

# --- Simulation and rendering ---
with mujoco.Renderer(model, height, width) as renderer:
    prev_com = data.subtree_com[0].copy()
    for i in range(n_frames):
        while data.time * framerate < i:
            # --- Open-loop gait ---
            data.ctrl[:] = amplitude * np.sin(2*np.pi*freq*t + phase_offsets)

            # --- Compute CoM and stabilization ---
            com = data.subtree_com[0]  # CoM of entire robot
            com_vel = (com - prev_com) / dt
            prev_com = com.copy()

            # Feedback torques based on CoM offsets
            # Roll stabilization: move opposite to lateral CoM deviation
            tau_roll  = -Kp*com[1] - Kd*com_vel[1]  # Y axis
            # Pitch stabilization: move opposite to forward/back CoM deviation
            tau_pitch = -Kp*com[0] - Kd*com_vel[0]  # X axis

            # Apply torques to first two actuators (assumes torso orientation joints)
            data.ctrl[0] += tau_roll
            data.ctrl[1] += tau_pitch

            # Step physics
            mujoco.mj_step(model, data)
            t += dt

        # Render frame
        renderer.update_scene(data)
        frame = renderer.render()
        frames.append(frame)

# --- Show video ---
media.show_video(frames, fps=framerate)


This browser does not support the video tag.


# COM + stability margin

In [32]:
import mujoco
import numpy as np
import mediapy as media
from scipy.spatial import ConvexHull
from shapely.geometry import Point, Polygon

# --- Load model ---
model = mujoco.MjModel.from_xml_path("ant.xml")
data = mujoco.MjData(model)

# --- Gait parameters ---
t = 0.0
dt = model.opt.timestep
amplitude = 0.8   # torque amplitude for legs
freq = 1        # gait frequency
phase_offsets = np.array([0, np.pi, np.pi, 0, np.pi, 0, 0, np.pi])  # trot gait

# --- Video setup ---
height, width = 240, 320
n_seconds = 20
framerate = 30
n_frames = n_seconds * framerate
frames = []

# --- Stabilization gains ---
Kp = 30.0
Kd = 10.0
desired_margin = 0.5  # minimal distance from edge of support polygon

# --- Foot names for support polygon ---
foot_names = ["front_left_foot", "front_right_foot", "rear_left_foot", "rear_right_foot"]
foot_ids = [mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, name) for name in foot_names]

# --- Simulation and rendering ---
with mujoco.Renderer(model, height, width) as renderer:
    prev_com = data.subtree_com[0].copy()
    for i in range(n_frames):
        while data.time * framerate < i:
            # --- Open-loop gait ---
            data.ctrl[:] = amplitude * np.sin(2*np.pi*freq*t + phase_offsets)

            # --- Center-of-mass ---
            com = data.subtree_com[0]
            com_vel = (com - prev_com) / dt
            prev_com = com.copy()

            # --- Support polygon & stability margin ---
            foot_positions = np.array([data.geom_xpos[fid][:2] for fid in foot_ids])
            
            # Only compute hull if there are 3 or more unique points
            if len(np.unique(foot_positions, axis=0)) >= 3:
                hull = ConvexHull(foot_positions)
                polygon = foot_positions[hull.vertices]
            else:
                hull = None
                polygon = foot_positions  # fallback, or skip stability computation
            poly = Polygon(polygon)
            com_xy = com[:2]
            point = Point(com_xy)

            if poly.contains(point):
                stability_margin = poly.exterior.distance(point)
            else:
                stability_margin = -poly.exterior.distance(point)

            # --- Feedback: move torso to maintain stability ---
            centroid = np.mean(polygon, axis=0)
            error = com_xy - centroid
            if stability_margin < desired_margin:
                tau_roll  = -Kp * error[1] - Kd * com_vel[1]   # lateral
                tau_pitch = -Kp * error[0] - Kd * com_vel[0]  # forward/back
                data.ctrl[0] += tau_roll
                data.ctrl[1] += tau_pitch

            # Step physics
            mujoco.mj_step(model, data)
            t += dt

        # Render frame
        renderer.update_scene(data)
        frame = renderer.render()
        frames.append(frame)

# --- Play video ---
media.show_video(frames, fps=framerate)


This browser does not support the video tag.


# gymnasium

In [5]:
env = gym.make("Ant-v4", render_mode="human")
obs, _ = env.reset()

for _ in range(1000):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)

    # Access mujoco data
    sim = env.unwrapped.data  
    model = env.unwrapped.model  

    # Body masses and COM positions
    masses = model.body_mass  # shape (num_bodies,)
    xpos = sim.xipos           # (num_bodies, 3), center of mass of each body in world frame

    # Full system CoM (mass-weighted average)
    com = np.sum(masses[:, None] * xpos, axis=0) / np.sum(masses)
    print("Center of Mass:", com)

    if terminated or truncated:
        obs, _ = env.reset()

env.close()

/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


Center of Mass: [0.06518301 0.0165754  0.78952905]
Center of Mass: [0.06286259 0.01395017 0.74475701]
Center of Mass: [0.06061275 0.01119042 0.67535908]
Center of Mass: [0.0475038  0.01386206 0.59668943]
Center of Mass: [-0.01099248  0.03685179  0.55391807]
Center of Mass: [-0.05730533  0.02147907  0.54243385]
Center of Mass: [-0.09217784 -0.0095382   0.53097037]
Center of Mass: [-0.12742758 -0.03547822  0.50747191]
Center of Mass: [-0.16353581 -0.04646987  0.49442936]
Center of Mass: [-0.18479953 -0.05193303  0.47702513]
Center of Mass: [-0.16507838 -0.05552627  0.48822763]
Center of Mass: [-0.14016606 -0.05731236  0.48725304]
Center of Mass: [-0.13327761 -0.05509479  0.4818538 ]
Center of Mass: [-0.20670552 -0.05181208  0.55761094]
Center of Mass: [-0.2867877  -0.04761722  0.62356718]
Center of Mass: [-0.35127153 -0.02440104  0.69623771]
Center of Mass: [-4.16337575e-01 -5.70151467e-04  7.45172144e-01]
Center of Mass: [-0.48140876  0.02326676  0.76958371]
Center of Mass: [-0.54650035

/home/yifeng/work/env/mujoco/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


Center of Mass: [-1.69113257  1.13875971  0.32658334]
Center of Mass: [-1.69270041  1.16115293  0.32658053]
Center of Mass: [-1.69522156  1.18264346  0.32272626]
Center of Mass: [-1.69875338  1.20463913  0.32023855]
Center of Mass: [-1.71384569  1.22214346  0.32875892]
Center of Mass: [-1.73231912  1.23642707  0.32165756]
Center of Mass: [-1.74430853  1.23871136  0.32378922]
Center of Mass: [-1.75632633  1.23704156  0.31891808]
Center of Mass: [-1.76954145  1.24290566  0.3138314 ]
Center of Mass: [-1.78250076  1.25352833  0.31513908]
Center of Mass: [-1.79205316  1.24235546  0.34191501]
Center of Mass: [-1.80210036  1.22815527  0.34820091]
Center of Mass: [-1.81213269  1.2139413   0.32998215]
Center of Mass: [-1.81740797  1.20387718  0.31871241]
Center of Mass: [-1.81212627  1.195339    0.3215573 ]
Center of Mass: [-1.80561127  1.18247953  0.3115565 ]
Center of Mass: [-1.80840318  1.17533792  0.31052429]
Center of Mass: [-1.81417792  1.16823105  0.310781  ]
Center of Mass: [-1.81950856

# Summary
Why stability margin and COM adjustment likely won’t work

Ant actuators are all leg joints.

data.ctrl[0] and data.ctrl[1] are knee/hip actuators of one leg.

They cannot directly control torso orientation.

Applying torques here might just move the legs weirdly without correcting the torso.

No torso actuators in standard Ant

The base is a free-floating body with no motors; you cannot directly apply torque to the torso unless you modify the XML.

Feedback is applied to wrong DOFs

To stabilize the torso using this kind of PD control, you’d need torque-capable joints connecting the torso to the legs in pitch/roll, which the standard Ant doesn’t have.

Proper sine-wave gait with phase offsets is often enough for stable forward motion.